In [ ]:
import pandas as pd
import re
average_path_1 = "tpc-ds/result_log/result_log_1GB/time_1/average_query_sys_params.csv"
average_path_2 = "tpc-ds/result_log/result_log_1GB/time_1/average_query_sys_params.csv"
average_path_3 = "tpc-ds/result_log/result_log_1GB/time_1/average_query_sys_params.csv"

worst_path_1 = "tpc-ds/result_log/result_log_1GB/time_1/worst_query_sys_params.csv"
worst_path_2 = "tpc-ds/result_log/result_log_1GB/time_1/worst_query_sys_params.csv"
worst_path_3 = "tpc-ds/result_log/result_log_1GB/time_1/worst_query_sys_params.csv"

worst_output_path = "tpc-ds/result_log/result_log_1GB/time_1/worst_query_sys_params.csv"
average_output_path = "tpc-ds/result_log/result_log_1GB/time_1/average_query_sys_params.csv"
# Đường dẫn đến 3 file CSV
average_paths = [
    average_path_1,average_path_2,average_path_3
]
worst_paths = [
    worst_path_1,worst_path_2,worst_path_3
]
# Các cột cần thiết
columns_to_use = [
    "query_id","cpu_used(%)","ram_used(gb)","time(ms)"
]


In [ ]:
def process_query_metrics(paths, columns_to_use, output_path):
    # Read, filter, and clean data
    dfs = [pd.read_csv(path, comment='/')[columns_to_use].assign(query_id=lambda df: df['query_id'].str.strip()) 
           for path in paths]
    
    # Merge data and calculate average by query_id
    all_data = pd.concat(dfs)
    average_metrics = all_data.groupby('query_id', as_index=False).mean(numeric_only=True)
    average_metrics = average_metrics.round(2)
    
    # Add column for sorting by number in query_id
    average_metrics['query_number'] = average_metrics['query_id'].apply(lambda x: int(re.search(r'\d+', x).group()))
    
    # Sort by query number
    average_metrics = average_metrics.sort_values(by='query_number').drop(columns=['query_number'])
    
    # Write to file
    average_metrics.to_csv(output_path, index=False)
    
    return average_metrics

In [ ]:
# Process average metrics
average_metrics = process_query_metrics(
    average_paths, 
    columns_to_use, 
    'average_query_metrics_10gb.csv'
)

# Process worst metrics
worst_metrics = process_query_metrics(
    worst_paths,
    columns_to_use,
    'worst_query_metrics_10gb.csv'
)